In [9]:
### dimension reduce & cluster with UMAP

import umap
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

from io import BytesIO
from PIL import Image
import base64

from bokeh.plotting import figure, show, output_notebook, save
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.transform import factor_mark

In [10]:
## import features and charter listf

array_path = r"/home/tschernn/becore-clustering/img_array.pkl"

with open(array_path, 'rb') as file:
    img_array = pickle.load(file)

feature_path = r"/home/tschernn/becore-clustering/features_hog.pkl"

with open(feature_path, 'rb') as file:
    data = pickle.load(file)

charter_path = r"/home/tschernn/becore-clustering/charter_list.pkl"

with open(charter_path, 'rb') as file:
    charters = pickle.load(file)

data.shape

(304, 304)

In [11]:
img_vectors = [img.flatten() for img in img_array]
img_vectors = np.array(img_vectors)
img_vectors.shape

(304, 2359296)

In [12]:
## fit and transform

reducer = umap.UMAP()
embedding = reducer.fit_transform(data)

# UMAP by default reduces to 2D
embedding.shape

(304, 2)

In [13]:
reducer = umap.UMAP(a=None, angular_rp_forest=False, b=None,
     force_approximation_algorithm=False, init='spectral', learning_rate=1.0,
     local_connectivity=1.0, low_memory=False, metric='euclidean',
     metric_kwds=None, min_dist=0.1, n_components=2, n_epochs=None,
     n_neighbors=15, negative_sample_rate=5, output_metric='euclidean',
     output_metric_kwds=None, random_state=42, repulsion_strength=1.0,
     set_op_mix_ratio=1.0, spread=1.0, target_metric='categorical',
     target_metric_kwds=None, target_n_neighbors=-1, target_weight=0.5,
     transform_queue_size=4.0, transform_seed=42, unique=False, verbose=False)
reducer.fit(data.data)

embedding = reducer.transform(data.data)

/home/tschernn/.local/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [14]:
### export data as pickle file
feature_path = feature_path.split('.')[0]
feature_path = feature_path + '_dimred.pkl'

with open(feature_path,'wb') as file:
    pickle.dump(embedding, file)
    print(f'Saved features as {file.name}.')

Saved features as /home/tschernn/becore-clustering/features_hog_dimred.pkl.


In [15]:
### remove unneeded img_array dimensions

img_array = img_array[:, :, :, 0]
img_array.shape

(304, 1024, 768)

In [16]:
### bokeh implementation adapted from https://umap-learn.readthedocs.io/en/latest/basic_usage.html

def embeddable_image(data):
    img_data = 255 - 15 * data.astype(np.uint8)
    image = Image.fromarray(img_data, mode='L').resize((256, 256), Image.Resampling.BICUBIC)
    buffer = BytesIO()
    image.save(buffer, format='png')
    for_encoding = buffer.getvalue()
    return 'data:image/png;base64,' + base64.b64encode(for_encoding).decode()


## draw plot

output_notebook()

charters_df = pd.DataFrame(embedding, columns=('x', 'y'))

charters_df['class'] = [x[1].split('_')[-2] for x in charters]
charters_df['origin'] = [x[1].split('_')[-1] for x in charters]
charters_df['image'] = list(map(embeddable_image, img_array))
charters_df['name'] = [x[0] for x in charters]

plot_figure = figure(
    title = 'UMAP projection of the charters',
    width = 800,
    height = 800,
    tools = ('pan, wheel_zoom, reset')
)

plot_figure.add_tools(HoverTool(tooltips="""
<div>
    <div>
        <img src='@image' style='float: left; margin: 5px 5px 5px 5px'/>
    </div>
    <div>
        <span style='font-size: 16px; color: #224499'>Charter:</span>
        <span style='font-size: 18px'>@name</span>
    </div>
</div>
"""))

legend_entries = []

for class_name, color in zip(charters_df['class'].unique(), ['#8e0000', '#008000', '#00578e']):
    for origin_name, marker in zip(charters_df['origin'].unique(), ['asterisk', 'circle']):
        class_data = charters_df[charters_df['class'] == class_name]
        class_data = class_data[charters_df['origin'] == origin_name]
        plot_figure.scatter(
            'x',
            'y',
            source = class_data,
            color = color,
            marker = marker,
            line_alpha = 0.6,
            fill_alpha = 0.6,
            size = 7,
            legend_label = f'{class_name.title()} {origin_name.upper()}'
        )
        # Append legend entry
        legend_entries.append((f'{class_name.title()} {origin_name.upper()}', [plot_figure.renderers[-1]]))

save(plot_figure, filename='umap_plot.html')
#show(plot_figure)

Loading BokehJS ...

/tmp/ipykernel_2872670/2381987929.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  class_data = class_data[charters_df['origin'] == origin_name]
/tmp/ipykernel_2872670/2381987929.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  class_data = class_data[charters_df['origin'] == origin_name]
/tmp/ipykernel_2872670/2381987929.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  class_data = class_data[charters_df['origin'] == origin_name]
/tmp/ipykernel_2872670/2381987929.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  class_data = class_data[charters_df['origin'] == origin_name]
/tmp/ipykernel_2872670/2381987929.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  class_data = class_data[charters_df['origin'] == origin_name]
/tmp/ipykernel_2872670/2381987929.py:47: UserWarning: Boolean Series key will be reindexed to m

'/home/tschernn/becore-clustering/umap_plot.html'